In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/logicalErrorFix_CodeT5_no_LineNumber

/content/drive/MyDrive/logicalErrorFix-2


In [ ]:
!bash

bash: cannot set terminal process group (2620): Inappropriate ioctl for device
bash: no job control in this shell
/content/drive/MyDrive/logicalErrorFix-2# bash eval_codeT5.sh
02/28/2024 14:42:12 - INFO - numexpr.utils -   NumExpr defaulting to 2 threads.
02/28/2024 14:42:13 - INFO - __main__ -   Namespace(model_type='codeT5', model_name_or_path='Salesforce/codet5-base', output_dir='model/cpp', load_model_path='model/cpp/checkpoint-best-bleu-CodeT5/pytorch_model.bin', train_filename=None, dev_filename='./data/edit_distance/pair_code_edit_dist_valid.txt', test_filename='./data/edit_distance/pair_code_edit_dist_test.txt', config_name='', tokenizer_name='', max_source_length=512, max_target_length=64, do_train=False, do_eval=False, do_test=True, do_lower_case=False, no_cuda=False, train_batch_size=8, eval_batch_size=2, gradient_accumulation_steps=1, learning_rate=5e-05, beam_size=10, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, eval_steps=-1

In [ ]:
from __future__ import absolute_import
import os
import sys
import bleu
import pickle
import torch
import json
import random
import logging
import argparse
import numpy as np
import copy
from io import open
from itertools import cycle
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaModel, RobertaTokenizer)

from transformers import T5ForConditionalGeneration
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer),
                 'codeT5' : (None, T5ForConditionalGeneration, RobertaTokenizer)}

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

class Example(object):
    """A single training/test example."""
    def __init__(self,
                 idx,
                 source,
                 target,
                 ):
        self.idx = idx
        self.source = source
        self.target = target

import pandas as pd
COLUMNS = ['Correct_code', 'Incorrect_code', 'Statement']

def read_examples(filename):
  """Read examples from filename for DeepFix style training Line stmt Line stmt Line stmt ..."""
  examples = []
  data = pd.read_csv(filename, sep='\t', header=[0]).drop(columns=COLUMNS[0])
  for idx, elem in data.iterrows():
    code = ' '.join(elem[COLUMNS[1]].split('||| '))[:-1].strip()
    stmt = elem[COLUMNS[2]].strip()

    examples.append(
      Example(
              idx = idx,
              source = code,
              target = stmt,
              )
    )
  return examples

class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask


def process_code_examples(input_examples):
    def parse_and_continue_on_mismatch(code_str):
        code_dict = {}
        expected_line_number = 1
        parts = code_str.split()
        current_line = None
        for part in parts:
            if part.isdigit():
                line_number = int(part)
                if line_number == expected_line_number:
                    current_line = line_number
                    code_dict[current_line] = ""
                    expected_line_number += 1
            else:
                if current_line is not None:
                    code_dict[current_line] += part + " "
        for line in code_dict:
            code_dict[line] = code_dict[line].strip()
        return code_dict

    def remove_line_numbers(source_dict):
        ret = []
        for _, v in source_dict.items():
            ret.append(v)
        return ' '.join(ret)

    def parse_single_line_to_dict(line_str):
        parts = line_str.split(' ', 1)
        line_number = parts[0]
        code = parts[1] if len(parts) > 1 else ""
        return {int(line_number): code}

    def update_values_from_dict(dict1, dict2):
        for key in dict1:
            if key in dict2:
                dict1[key] = dict2[key]
        return dict1

    source_code_dict = {}
    for i, example in enumerate(input_examples):
        source_code_dict[i] = parse_and_continue_on_mismatch(example.source)

    target_dict = {}
    for i, example in enumerate(input_examples):
        target_dict[i] = parse_single_line_to_dict(example.target)

    target_code_dict = {i: update_values_from_dict(copy.deepcopy(source_code_dict[i]), target_dict[i]) for i in range(len(target_dict))}

    input_code = {}
    for i in range(len(source_code_dict)):
        input_code[i] = {
            "source": remove_line_numbers(source_code_dict[i]),
            "target": remove_line_numbers(target_code_dict[i])
        }

    return input_code

In [ ]:
import argparse

# 명령줄 인자를 직접 코드 내에서 설정
args = argparse.Namespace(
    model_type='roberta',
    model_name_or_path='roberta-base',
    train_filename='./data/edit_distance/pair_code_edit_dist_train.txt',
    dev_filename='./data/edit_distance/pair_code_edit_dist_val.txt',
    config_name='',
    tokenizer_name='',
    max_source_length=64,
    max_target_length=32,
    do_train=True,
    do_eval=True,
    do_test=True,
    do_lower_case=True,
    no_cuda=True,
    train_batch_size=8,
    eval_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    beam_size=10,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    num_train_epochs=3.0,
    max_steps=-1,
    eval_steps=-1,
    train_steps=-1,
    warmup_steps=0,
    local_rank=-1,
    seed=42,
)

# 예제 사용
print(args.model_type)  # 출력: roberta
print(args.do_train)    # 출력: True


roberta
True


In [ ]:
train_examples = read_examples(args.train_filename)

In [ ]:
train = process_code_examples(train_examples)

In [ ]:
print(train[0])


{'source': '#include <bits/stdc++.h> using namespace std; int ara[500008]; set<int> s; set<int>::iterator it; int n; void check(int num) { for (int i = 1; i < n; i++) { if (!(ara[i] % num == || ara[n + i] % num == 0)) { return; } } cout << num << endl; exit(0); } int primefactor(int num) { int i; for (i = 2; i <= num / i; i++) { bool flag = 0; while (num % i == 0) { num /= i; if (!flag) { check(i); } flag = 1; } } if (num > i) { check(num); } } int main() { scanf("%d", &n); for (int i = 0; i < n; i++) { scanf("%d%d", &ara[i], &ara[n + i]); } primefactor(ara[0]); primefactor(ara[n]); cout << "-1" << endl; return 0; }', 'target': '#include <bits/stdc++.h> using namespace std; int ara[500008]; set<int> s; set<int>::iterator it; int n; void check(int num) { for (int i = 1; i < n; i++) { if (!(ara[i] % num == || ara[n + i] % num == 0)) { return; } } cout << num << endl; exit(0); } int primefactor(int num) { int i; for (i = 2; i <= num / i; i++) { bool flag = 0; while (num % i == 0) { num /=

In [ ]:
for index, code, in zip(train.keys(), train.values()):
    print(index, code['source'])

0 #include <bits/stdc++.h> using namespace std; int ara[500008]; set<int> s; set<int>::iterator it; int n; void check(int num) { for (int i = 1; i < n; i++) { if (!(ara[i] % num == || ara[n + i] % num == 0)) { return; } } cout << num << endl; exit(0); } int primefactor(int num) { int i; for (i = 2; i <= num / i; i++) { bool flag = 0; while (num % i == 0) { num /= i; if (!flag) { check(i); } flag = 1; } } if (num > i) { check(num); } } int main() { scanf("%d", &n); for (int i = 0; i < n; i++) { scanf("%d%d", &ara[i], &ara[n + i]); } primefactor(ara[0]); primefactor(ara[n]); cout << "-1" << endl; return 0; }
1 #include <bits/stdc++.h> #pragma comment(linker, "/stack:200000000") #pragma GCC optimize("Ofast") #pragma GCC target("sse,sse2,sse3,ssse3,sse4,popcnt,abm,mmx,avx,tune=native") using namespace std; inline long long rint() { long long x = 0, f = 1; char c = getchar(); while (c < '0' || c > '9') { if (c == '-') f = -1; c = getchar(); } while (c >= '0' && c <= '9') { x = x * + c - '0'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 #include <bits/stdc++.h> using namespace std; int a[100005]; int main(void) { int n, i; scanf("%d", &n); for (i = 1; i <= n; i++) scanf("%d", &a[i]); if (n >= 55) printf("YES\n"); else { sort(a + 1, a + n + 1); for (i = 1; i <= n - 2; i++) { if (a[i] + a[i + 1] >= a[i + 2]) { printf("YES\n"); return 0; } } printf("NO\n"); } return 0; }
8039 #include <bits/stdc++.h> using namespace std; vector<long long int> vec; int main() { std::ios::sync_with_stdio(false); long long int n, i, a, flag = 0; cin >> n; for (i = 0; i < n; i++) { cin >> a; vec.push_back(a); } sort(vec.begin(), vec.end()); for (i = 0; i < vec.size() - 3; i++) { if (vec[i] + vec[i + 1] > vec[i + 2] && vec[i + 2] + vec[i + 1] > vec[i] && vec[i] + vec[i + 2] > vec[i + 1]) { flag = 1; break; } } if (flag == 1) { cout << "YES"; } else cout << "NO"; }
8040 #include <bits/stdc++.h> using namespace std; int main() { int n; cin >> n; vector<int> ar(n); for (int i = 0; i < n; i++) cin >> ar[i]; sort(ar.begin(), ar.end()); for (int i

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [ ]:
!bash

bash: cannot set terminal process group (2422): Inappropriate ioctl for device
bash: no job control in this shell
/content/drive/MyDrive/logicalErrorFix-2# bash train_codeT5.sh
02/27/2024 03:03:05 - INFO - numexpr.utils -   NumExpr defaulting to 2 threads.
02/27/2024 03:03:05 - INFO - __main__ -   Namespace(model_type='codeT5', model_name_or_path='Salesforce/codet5-base', output_dir='model/cpp', load_model_path=None, train_filename='./data/edit_distance/pair_code_edit_dist_train.txt', dev_filename='./data/edit_distance/pair_code_edit_dist_valid.txt', test_filename=None, config_name='', tokenizer_name='', max_source_length=512, max_target_length=64, do_train=True, do_eval=True, do_test=False, do_lower_case=False, no_cuda=False, train_batch_size=4, eval_batch_size=4, gradient_accumulation_steps=1, learning_rate=5e-05, beam_size=10, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, eval_steps=1000, train_steps=50000, warmup_steps=0, local_rank=-1

In [ ]:
import transformers

print(f"transformers version: {transformers.__version__}")

In [3]:
!bash

bash: cannot set terminal process group (1210): Inappropriate ioctl for device
bash: no job control in this shell
/content/drive/MyDrive/logicalErrorFix-2# bash train_codeT5.sh
03/04/2024 17:23:25 - INFO - numexpr.utils -   NumExpr defaulting to 2 threads.
03/04/2024 17:23:25 - INFO - __main__ -   Namespace(model_type='codeT5', model_name_or_path='Salesforce/codet5-base', output_dir='model/cpp', load_model_path=None, train_filename='./data/edit_distance/pair_code_edit_dist_train.txt', dev_filename='./data/edit_distance/pair_code_edit_dist_valid.txt', test_filename=None, config_name='', tokenizer_name='', max_source_length=512, max_target_length=64, do_train=True, do_eval=True, do_test=False, do_lower_case=False, no_cuda=False, train_batch_size=4, eval_batch_size=4, gradient_accumulation_steps=1, learning_rate=5e-05, beam_size=10, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, eval_steps=1000, train_steps=50000, warmup_steps=0, local_rank=-1